# PodcastFetch Analysis

This notebook is for analyzing and working with podcast data.


In [ ]:
# Import standard library
import os
import sqlite3
from pathlib import Path

# Import third-party libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import PodcastFetch modules
from podcast_fetch import config
from podcast_fetch import (
    # Data functions
    normalize,
    collect_data,
    summarise_podcasts,
    get_rss_from_apple_podcast,
    normalize_feed_url,
    get_podcast_title,
    get_apple_podcast_info,
    # Database functions
    is_valid_database,
    get_db_connection,
    clean_dataframe_for_sqlite,
    table_exists,
    summary_exists,
    has_downloaded_episodes,
    verify_downloaded_files_exist,
    add_download_columns_to_table,
    update_download_info,
    update_all_tables_with_download_columns,
    add_podcast_image_url_to_summary,
    add_indexes_to_table,
    update_all_tables_with_indexes,
    explain_query_plan,
    # Download functions
    sanitize_filename,
    show_podcast_summary,
    download_all_episodes,
    download_last_episode,
    update_summary,
)

# Set up plotting style
plt.style.use(config.PLOTTING_STYLE)
sns.set_palette(config.SEABORN_PALETTE)

print("Libraries imported successfully!")
print(f"Using database: {config.DB_PATH}")
print(f"Downloads folder: {config.DOWNLOADS_FOLDER}")


In [ ]:
# Read the feeds.txt file and save it as a list of strings
# Automatically handles both RSS feed URLs and Apple Podcast links
# - RSS feed URLs are used directly
# - Apple Podcast links are automatically converted to RSS feed URLs
# - Podcast titles are fetched and displayed for each feed
with open(config.FEEDS_FILE, 'r') as file:
    feeds = file.readlines()

# Strip newlines and normalize feed URLs, getting podcast titles
normalized_feeds = []
feed_info = []  # Store (rss_url, title) tuples
apple_podcast_count = 0
rss_feed_count = 0

for feed in feeds:
    feed = feed.strip()
    if not feed or feed.startswith('#'):  # Skip empty lines and comments
        continue
    
    try:
        # Check if it's an Apple Podcast link
        is_apple_podcast = 'podcasts.apple.com' in feed or 'itunes.apple.com' in feed
        
        if is_apple_podcast:
            # Convert Apple Podcast link to RSS feed and get title
            print(f"📱 Detected Apple Podcast link, converting to RSS feed...")
            print(f"   Apple Podcast: {feed}")
            try:
                rss_url, podcast_title = get_apple_podcast_info(feed)
                normalized_feeds.append(rss_url)
                feed_info.append((rss_url, podcast_title))
                print(f"   ✓ RSS Feed: {rss_url}")
                print(f"   📻 Podcast Title: {podcast_title}\n")
                apple_podcast_count += 1
            except Exception as e:
                # Fallback to regular conversion if getting info fails
                print(f"   ⚠️  Could not get podcast info, using basic conversion...")
                rss_url = normalize_feed_url(feed)
                podcast_title = get_podcast_title(rss_url)
                normalized_feeds.append(rss_url)
                feed_info.append((rss_url, podcast_title))
                print(f"   ✓ RSS Feed: {rss_url}")
                print(f"   📻 Podcast Title: {podcast_title}\n")
                apple_podcast_count += 1
        else:
            # Already an RSS feed URL, use as-is and get title
            normalized_feed = normalize_feed_url(feed)  # Still normalize to validate
            podcast_title = get_podcast_title(normalized_feed)
            normalized_feeds.append(normalized_feed)
            feed_info.append((normalized_feed, podcast_title))
            rss_feed_count += 1
            
    except Exception as e:
        print(f"⚠️  Warning: Error processing feed URL '{feed}': {e}")
        print(f"   Using original URL as-is (may cause issues if invalid)\n")
        normalized_feeds.append(feed)  # Use original URL if conversion fails
        feed_info.append((feed, "Unknown Podcast"))

feeds = normalized_feeds
print(f"{'='*60}")
print(f"📊 Feed Summary:")
print(f"   Total feeds loaded: {len(feeds)}")
if apple_podcast_count > 0:
    print(f"   Apple Podcast links converted: {apple_podcast_count}")
if rss_feed_count > 0:
    print(f"   RSS feed URLs (direct): {rss_feed_count}")
print(f"{'='*60}\n")
print(f"📻 Podcasts Loaded:")
for i, (rss_url, title) in enumerate(feed_info, 1):
    print(f"  {i}. {title}")
    print(f"     RSS Feed: {rss_url}\n")

In [ ]:
# All functions are now imported from podcast_fetch package
# Functions available:
# - normalize(name)
# - collect_data(feed)
# - summarise_podcasts(df)
# - get_rss_from_apple_podcast(apple_url) - Convert Apple Podcast link to RSS feed
# - normalize_feed_url(feed_url) - Automatically detect and convert Apple Podcast links
# - And many more...

# Example usage:
# 1. Convert an Apple Podcast link to RSS feed URL:
#    apple_url = "https://podcasts.apple.com/us/podcast/example/id123456789"
#    rss_url = get_rss_from_apple_podcast(apple_url)
#
# 2. Or use normalize_feed_url() which automatically handles both:
#    - Apple Podcast links → converts to RSS feed
#    - RSS feed URLs → returns as-is
#    feed_url = normalize_feed_url(any_feed_url)

print("All PodcastFetch functions are available via imports!")
print("\n💡 Tip: In feeds.txt, you can mix both types:")
print("   - Direct RSS feed URLs (e.g., https://example.com/feed.rss)")
print("   - Apple Podcast links (e.g., https://podcasts.apple.com/.../id123456789)")
print("   Both will be automatically handled!")

In [ ]:
# Function moved to podcast_fetch.data.collection module
# Available as: collect_data(feed)
# This cell can be removed or used for testing
print("collect_data() is available from podcast_fetch package")


In [ ]:
# Function moved to podcast_fetch.data.summary module
# Available as: summarise_podcasts(df)
# This cell can be removed or used for testing
print("summarise_podcasts() is available from podcast_fetch package")


In [ ]:
# All database utility functions are now in podcast_fetch.database module
# Available functions:
# - clean_dataframe_for_sqlite(df)
# - is_valid_database(db_path)
# - table_exists(conn, table_name)
# - summary_exists(conn, podcast_name)
# - add_download_columns_to_table(conn, table_name)
# - update_download_info(conn, podcast_name, episode_id, file_path)
# - update_all_tables_with_download_columns(conn)
print("All database utility functions are available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.database.connection module
# Available as: is_valid_database(db_path)
print("is_valid_database() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.database.queries module
# Available as: table_exists(conn, table_name)
print("table_exists() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.database.queries module
# Available as: summary_exists(conn, podcast_name)
print("summary_exists() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.database.schema module
# Available as: add_download_columns_to_table(conn, table_name)
print("add_download_columns_to_table() is available from podcast_fetch package")


In [ ]:
# Function moved to podcast_fetch.database.schema module
# Available as: update_download_info(conn, podcast_name, episode_id, file_path)
print("update_download_info() is available from podcast_fetch package")


In [ ]:
# Function moved to podcast_fetch.database.schema module
# Available as: update_all_tables_with_download_columns(conn)
print("update_all_tables_with_download_columns() is available from podcast_fetch package")

# Run this to update existing tables (optional - new tables will have these columns automatically)
# update_all_tables_with_download_columns(conn)

# Also available: add_indexes_to_table(conn, table_name) and update_all_tables_with_indexes(conn)
# Indexes are automatically added when creating new tables, but you can update existing tables:
# update_all_tables_with_indexes(conn)


## Code Organization

All functions have been extracted to the `podcast_fetch` Python package.

**Module Structure:**
- `podcast_fetch.config` - Configuration constants
- `podcast_fetch.data` - Data collection and processing
- `podcast_fetch.database` - Database utilities
- `podcast_fetch.download` - Download operations

All functions are imported in Cell 1 and available for use.


In [ ]:
# All functions are now in the podcast_fetch package
# No manual updates needed - functions are already updated in the modules
print("All functions are available from podcast_fetch package!")


In [ ]:
# Check if the database already exists and is valid
db_path = config.DB_PATH
db_exists = os.path.exists(db_path)

if db_exists:
    if is_valid_database(db_path):
        print(f"Database '{db_path}' already exists and is valid. Connecting to existing database.")
    else:
        print(f"⚠️  Database '{db_path}' exists but is corrupted or invalid.")
        print(f"Creating a new database file...")
        # Backup the corrupted file
        backup_path = f"{db_path}.corrupted_backup"
        if os.path.exists(backup_path):
            os.remove(backup_path)
        os.rename(db_path, backup_path)
        print(f"Corrupted database backed up to: {backup_path}")
        db_exists = False

if not db_exists:
    print(f"Database '{db_path}' does not exist. Creating new database.")

# Connect to the database
# Note: For notebook use, we keep the connection open across cells
# For production scripts, use the context manager pattern:
#   with get_db_connection(db_path) as conn:
#       # your code here
#   # connection automatically closes

# For notebook convenience, create a connection that stays open
conn = sqlite3.connect(db_path, timeout=config.DB_TIMEOUT)
print(f"Database connection established.")
print(f"Note: Use 'conn.close()' when done, or use get_db_connection() context manager in scripts.")

In [ ]:
# Verify connection is still open and valid
try:
    conn.execute("SELECT 1")
except (sqlite3.ProgrammingError, sqlite3.OperationalError) as e:
    print(f"⚠️  Database connection error: {e}")
    print("Reconnecting to database...")
    db_path = config.DB_PATH
    conn = sqlite3.connect(db_path, timeout=config.DB_TIMEOUT)
    print("✓ Database reconnected successfully.")

for feed_url in feeds:
    # Strip newline characters from the feed URL
    feed_url = feed_url.strip()
    
    # Collect data from the feed
    df = collect_data(feed_url)
    
    # Get the title from the dataframe (it's stored in the 'author' column)
    title = df['author'].iloc[0] if len(df) > 0 else 'unknown'

    # Verificar si el podcast ya existe y tiene descargas antes de reemplazar
    should_skip = False
    try:
        if table_exists(conn, title) and summary_exists(conn, title):
            print(f"\n{'='*60}")
            print(f"Podcast '{title}' ya existe en la base de datos.")
            
            # Verificar si hay episodios descargados
            has_downloads = has_downloaded_episodes(conn, title)
            
            if has_downloads:
                # Verificar si los archivos existen en disco
                all_exist, total_downloaded, files_found = verify_downloaded_files_exist(conn, title)
                
                print(f"  📊 Estado del podcast:")
                print(f"    - Episodios descargados en BD: {total_downloaded}")
                print(f"    - Archivos encontrados en disco: {files_found}")
                
                if all_exist and total_downloaded > 0:
                    print(f"  ✓ Todos los archivos descargados existen en disco.")
                    print(f"  ⚠️  No se reemplazará la tabla para evitar perder datos de descargas.")
                    print(f"  💡 Si deseas actualizar, elimina manualmente la tabla '{title}' primero.")
                    print(f"{'='*60}\n")
                    should_skip = True
                elif files_found < total_downloaded:
                    print(f"  ⚠️  Algunos archivos no existen en disco ({files_found}/{total_downloaded}).")
                    print(f"  🔄 Se actualizará la tabla para sincronizar el estado.")
                    print(f"{'='*60}\n")
                else:
                    print(f"  ℹ️  No hay archivos en disco pero hay registros en BD.")
                    print(f"  🔄 Se actualizará la tabla.")
                    print(f"{'='*60}\n")
            else:
                print(f"  ℹ️  No hay episodios descargados.")
                print(f"  🔄 Se actualizará la tabla normalmente.")
                print(f"{'='*60}\n")
    except (sqlite3.OperationalError, sqlite3.ProgrammingError) as e:
        print(f"⚠️  Error checking podcast status: {e}")
        print("Attempting to reconnect...")
        db_path = config.DB_PATH
        conn = sqlite3.connect(db_path, timeout=config.DB_TIMEOUT)
        print("✓ Reconnected. Skipping verification for this podcast.")
    
    # Si hay descargas completas, saltar este podcast
    if should_skip:
        continue
    
    # Check if the table already exists before saving
    if table_exists(conn, title):
        print(f"Table '{title}' already exists. Replacing with new data.")
    else:
        print(f"Table '{title}' does not exist. Creating new table.")
    
    # Clean and save the podcast data
    try:
        df_clean = clean_dataframe_for_sqlite(df)
        df_clean.to_sql(title, conn, if_exists='replace', index=False)
        # Add performance indexes after creating/replacing table
        add_indexes_to_table(conn, title)
    except (sqlite3.OperationalError, sqlite3.ProgrammingError) as e:
        print(f"⚠️  Error saving podcast data: {e}")
        print("Attempting to reconnect...")
        db_path = config.DB_PATH
        conn = sqlite3.connect(db_path, timeout=config.DB_TIMEOUT)
        df_clean = clean_dataframe_for_sqlite(df)
        df_clean.to_sql(title, conn, if_exists='replace', index=False)
        # Add performance indexes after creating/replacing table
        add_indexes_to_table(conn, title)
        print("✓ Data saved after reconnection.")
    
    # Summarize the podcasts
    df_2 = summarise_podcasts(df)
    
    # Check if summary for this podcast already exists
    try:
        if summary_exists(conn, title):
            print(f"Summary for '{title}' already exists. Updating existing summary.")
            # Delete the old summary row before inserting the new one
            cursor = conn.cursor()
            cursor.execute("DELETE FROM summary WHERE name=?", (title,))
            conn.commit()
            # Now append the new summary
            df_2_clean = clean_dataframe_for_sqlite(df_2)
            df_2_clean.to_sql('summary', conn, if_exists='append', index=False)
        else:
            print(f"Summary for '{title}' does not exist. Creating new summary.")
            # Clean and save the summary (append mode to accumulate summaries from all podcasts)
            df_2_clean = clean_dataframe_for_sqlite(df_2)
            df_2_clean.to_sql('summary', conn, if_exists='append', index=False)
    except (sqlite3.OperationalError, sqlite3.ProgrammingError) as e:
        print(f"⚠️  Error updating summary: {e}")
        print("Attempting to reconnect...")
        db_path = config.DB_PATH
        conn = sqlite3.connect(db_path, timeout=config.DB_TIMEOUT)
        if summary_exists(conn, title):
            cursor = conn.cursor()
            cursor.execute("DELETE FROM summary WHERE name=?", (title,))
            conn.commit()
        df_2_clean = clean_dataframe_for_sqlite(df_2)
        df_2_clean.to_sql('summary', conn, if_exists='append', index=False)
        print("✓ Summary updated after reconnection.")
    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        raise


In [ ]:
# Function moved to podcast_fetch.download.utils module
# Available as: sanitize_filename(filename)
print("sanitize_filename() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.download.utils module
# Available as: show_podcast_summary(conn, podcast_names)
print("show_podcast_summary() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.download.downloader module
# Available as: download_all_episodes(conn, podcast_name, downloads_folder=None, delay_seconds=None)
# Uses config defaults if parameters are None
print("download_all_episodes() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.download.downloader module
# Available as: download_last_episode(conn, podcast_name, downloads_folder=None)
# Uses config defaults if parameters are None
print("download_last_episode() is available from podcast_fetch package")

In [ ]:
# Function moved to podcast_fetch.download.metadata module
# Available as: update_summary(conn, podcast_name)
print("update_summary() is available from podcast_fetch package")


In [ ]:
# Example: Download all episodes for a podcast
# Replace 'cosas_de_internet' with the podcast name you want to download from
# download_all_episodes(conn, 'cosas_de_internet')

# Or download all episodes for all podcasts
# First, collect all podcast names
podcast_names = []
for feed_url in feeds:
    df = collect_data(feed_url)
    title = df['author'].iloc[0] if len(df) > 0 else 'unknown'
    podcast_names.append(title)

# Show summary of all podcasts
if podcast_names:
    summary = show_podcast_summary(conn, podcast_names)
    
    # Ask for overall confirmation
    total = sum(summary.values())
    if total > 0:
        print(f"\n{'='*70}")
        user_input = input(f"Do you want to proceed with downloading {total} episode(s) across {len(podcast_names)} podcast(s)? (yes/no): ").strip().lower()
        print(f"{'='*70}\n")
        
        if user_input in ['yes', 'y']:
            # Download episodes for each podcast
            for podcast_name in podcast_names:
                if summary.get(podcast_name, 0) > 0:
                    download_all_episodes(conn, podcast_name)
        else:
            print("Download cancelled by user.")
    else:
        print("All episodes are already downloaded!")

# When done with all operations, close the connection
conn.close()
